In [1]:
class CulturalAlgorithm:
    def __init__(self, model, population_size, feature_range, p_accept, iter_count):
        self.model = model
        self.population_size = population_size
        self.feature_range = feature_range
        self.p_accept = p_accept
        self.iter_count = iter_count
        
        self.population = []
        for i in range(self.population_size):
            self.population.append([random.uniform(self.feature_range[0], self.feature_range[1]) 
                                     for _ in range(self.model.count_params())])
        
        self.best_solution = None
        self.best_fitness = float('-inf')
        
    def _evaluate_solution(self, solution):
        self.model.set_weights(solution)
        loss, accuracy = self.model.evaluate(X_valid, y_valid, verbose=0)
        return accuracy
    
    def _update_best_solution(self, solution, fitness):
        if fitness > self.best_fitness:
            self.best_solution = solution
            self.best_fitness = fitness
            
    def evolve(self):
        for i in range(self.iter_count):
            selected_solution = random.choice(self.population)
            new_solution = []
            for j, weight in enumerate(selected_solution):
                new_weight = weight + random.uniform(-0.5, 0.5)
                if new_weight < self.feature_range[0]:
                    new_weight = self.feature_range[0]
                elif new_weight > self.feature_range[1]:
                    new_weight = self.feature_range[1]
                new_solution.append(new_weight)
            
            new_fitness = self._evaluate_solution(new_solution)
            self._update_best_solution(new_solution, new_fitness)
            
            accept_prob = min(1, self.p_accept * (new_fitness / self.best_fitness))
            if random.uniform(0, 1) < accept_prob:
                self.population.append(new_solution)
            
            self.population = sorted(self.population, key=lambda x: self._evaluate_solution(x), reverse=True)[:self.population_size]
            
        self.model.set_weights(self.best_solution)
        
        return self.best_solution
